Project Ideas


suggested project: analyse wind speed around the country with a view to a wind farm


## Project Plan

__Research wind farms in Ireland__

- where are they usually located?
- what wind conditions do they need? any other site considerations? Rural locations.
- how much electricity do they generate? summer vs winter?
- cost?
- lifespan?
- advantages
- disadvantages
- anything else?

__Project questions__

What's the relationship between wind speed and power generated? Does the wind direction affect power generation? 

Is there a trend in wind speed? Is Ireland getting winder? Variations across the year? Time of day?

Is the technology in wind turbines improving? Is more electricity being generated for the same wind speed?

Does rain/temperature/anything affect the output? 

What happens during a storm? Does amount of wind generated electricity decrease/increase? 

Predict power output for wind farms in Ireland for the next week. Tricky

As I have weather information could solar power to fill the gaps when wind speeds are low? Probably too big a task for this project. 


__Find data__

Weather data from met Éireann historical data.
    can select by site, perhaps initially analyse data for a number of weather stations near a wind farm and also weather stations not near a wind farm. From the data can I see why that site was selected?

Eirgrid real-time system information on a daily basis. https://www.eirgrid.ie/grid/real-time-system-information They only show one day at a time. Can view information up to one month ago. Found a Github repository Daniel-Parke, wrote a python file to download all the data. He has raw files for actual amount of electricity produced by wind, actual demand and actual generation by every 15 min, year from 2014. Data for 2024 isn't complete. Will need to run the program to get 2024 data. Therefore can compare wind speeds and generation from 2014. KEEP Looking - there might be a more official source.

Is there much variation in wind across the country? Eirgrid data for entire country. 

## Introduction

Background information

## The Data

About the data set. 

## Organising and Cleaning the Data

Download the eirgrid csv files from the GitHub. 

Merge data. Write a function to do this. Would be convenient to have all the data in one large data set. Need to research working with large data sets. More difficult to load than smaller data sets.

Clean data

In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

First task is to merge the files vertically. Use pd.concat

research glob

https://saturncloud.io/blog/how-to-read-multiple-csv-files-into-python-pandas-dataframe/#:~:text=We%20use%20the%20glob%20function,the%20read_csv%20function%20from%20Pandas.

In [ ]:
# Load the wind electricity files

csv_files = glob.glob('data/electricity/*.csv')

# Create an empty dataframe to store the combined data
electricity_df = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file, 
                     header = None, 
                     names = ['date', 'wind_actual', 'location', 'wind_value'], 
                     index_col= 'date',
                     usecols= ['date', 'wind_value'])
    electricity_df = pd.concat([electricity_df, df])

electricity_df.head(20)

,wind_value
date,
01-Jan-2014 00:00:00,1020.0
01-Jan-2014 00:15:00,995.0
01-Jan-2014 00:30:00,933.0
01-Jan-2014 00:45:00,959.0
01-Jan-2014 01:00:00,921.0
01-Jan-2014 01:15:00,931.0
01-Jan-2014 01:30:00,882.0
01-Jan-2014 01:45:00,923.0
01-Jan-2014 02:00:00,919.0


In [18]:
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 361152 entries, 01-Jan-2014 00:00:00 to 01-Jan-2024 21:45:00
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   wind_value  361004 non-null  float64
dtypes: float64(1)
memory usage: 5.5+ MB


In [19]:
electricity_df.isna().sum()

wind_value    148
dtype: int64

## Exploratory Data Analysis

## Discussion of the Analysis

nice plots

## Machine Learning

some predictive analysis.

## Conclusion



## References

### Data Sets

[GitHub Daniel Parke]https://github.com/Daniel-Parke/EirGrid_Data_Download/tree/main
